# Splitting Data into Test-Train-Validation Sets

In [1]:
import csv
import argparse
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import OneHotEncoder

In [2]:
batting_data_path = 'data/Batting.csv'
# INFO:
# 101,332 Players with up to 20 features each (exluding year, including team)
# if metric not reported for player, set to 0.0 by default

df = pd.read_csv(batting_data_path, index_col = 0)
df = df.fillna(0)

In [9]:
df_recent_players = df[df.yearID >= 1975] #48k players
team_set = set(df_recent_players.teamID)
df_recent_players

#  NOTE: Potentially create one-hot encoding of teams...
## I chose to leave it off for now because I think it may
## create Data leakage in our model


,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,1975,1,ML4,AL,137,465.0,45.0,109.0,16.0,2.0,...,60.0,0.0,1.0,70.0,51.0,3.0,1.0,1.0,6.0,15.0
abbotgl01,1975,1,OAK,AL,30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acostcy01,1975,1,PHI,NL,6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adamsgl01,1975,1,SFN,NL,61,90.0,10.0,27.0,2.0,1.0,...,15.0,1.0,0.0,11.0,25.0,0.0,1.0,0.0,1.0,1.0
alburvi01,1975,1,MIN,AL,33,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
alexado01,1975,1,BAL,AL,32,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alexaga01,1975,1,SFN,NL,3,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
alexama01,1975,1,OAK,AL,63,10.0,16.0,1.0,0.0,0.0,...,0.0,17.0,10.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
allendi01,1975,1,PHI,NL,119,416.0,54.0,97.0,21.0,3.0,...,62.0,11.0,2.0,58.0,109.0,4.0,2.0,1.0,4.0,19.0


In [10]:
#  Randomly shuffle by player year
## NOTE: This may be an inaccurate assumption to make.
## We're implicitly assuming that training on players from 1975
## won't impact prediction accuracy for modern day players.

## If this is an incorrect assumption, consider data augmentation,
## taking into account differences between current-day MLB players

df_recent_players = df_recent_players.sample(frac=1) #randomly permute by year
player_ids = [df.iloc[:,0:0]]
df_recent_players
# player_ids = set(df_recent_players.playerID) #List of all player ids

# len(player_ids)

,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,,,,
garzama01,2008,1,TBA,AL,30,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0
stanlfr01,1982,1,OAK,AL,101,228.0,33.0,44.0,7.0,0.0,...,17.0,0.0,1.0,29.0,32.0,0.0,1.0,1.0,0.0,4.0
choji01,1998,1,BOS,AL,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pashnla01,1984,1,MIN,AL,13,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
morrijo07,1991,1,PHI,NL,85,127.0,15.0,28.0,2.0,1.0,...,6.0,2.0,0.0,12.0,25.0,4.0,1.0,0.0,0.0,1.0
landrti01,1987,2,LAN,NL,51,67.0,8.0,16.0,3.0,0.0,...,4.0,1.0,1.0,3.0,16.0,0.0,1.0,1.0,0.0,0.0
vinceni01,2012,1,SDN,NL,27,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
perrypa02,1987,2,CIN,NL,12,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bradlmi01,2006,1,OAK,AL,96,351.0,53.0,97.0,14.0,2.0,...,52.0,10.0,2.0,51.0,65.0,1.0,2.0,0.0,1.0,13.0


In [51]:
x_data = df_recent_players[df_recent_players.columns[6:]]